In [1]:
import ee
import datetime
import os
import itertools
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Define_Basic_Parameters

In [8]:
# define the num which represent the true number of verified sample-point sets
num = 5


# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))[-num:]
year_name_all  = list([f'{span[0]}_{span[1]}' for span in zip(range(1987,2020,3),range(1989,2020,3)) ])
year_name = year_name_all[-num:]

In [9]:
year_range

[('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

In [10]:
year_name

['2005_2007', '2008_2010', '2011_2013', '2014_2016', '2017_2019']

##### Classify Sample_with_img_value

In [11]:
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value'

Sample_with_value = [ee.FeatureCollection(f'{path}/Sample_{year}_extract_Landsat_Fourier_Normalized_img')
                    for year in year_name]

In [31]:
# get necessary img and verified points
Verified_point = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value/Sample_2017_2019_extract_Landsat_Fourier_Normalized_img")
input_band = ['B1', 'B10', 'B11', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']


# Instatiate the class with a name.
test = Classification(year_name ='2017_2019',
                      Input_img  = 0,
                      input_band = input_band,
                      Verified_point = Verified_point)


# perform the classification on Input_img and Samples
test.Stp_2_Classification_on_Samples()

# Get the classified samples
train_sample_classified        = test.Train_sample_classification
test_sample_classified         = test.Test_sample_classification

In [34]:
Accuracy_assesment(test_sample_classified).Stp_1_Calculate_Accuracy()

{'Overall_ACC': 91.76,
 'Precisioin_non_built': 93.13,
 'Precisioin_built': 77.39,
 'Recall_non_built': 97.74,
 'Recall_built': 51.79}